Running sequence:
1.   Code Block 1
2.   Code Block 2
3.   Code Block 3
4.   Code Block 4
5.   Code Block 5
6.   Code Block 6
7.   Code Block 7
8.   Code Block 8
9.   Code Block 9
10.  Code Block 10
11.  Code Block 11
12.  Code Block 12
13.  Code Block 13
14.  Code Block 14
15.  Code Block 16
16.  Code Block 17
17.  Code Block 18
18.  Code Block 19
19.  Code Block 21
20.  Code Block 22

Note: Code Block 14 is the Cross Validation function, Code Block 15 and Code Block 20 used this Cross Validation Function.


In [1]:
'''
Code Block 1
Global libraries
'''
from pathlib import Path
import glob
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [2]:
'''
Code Block 2
'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
'''
Code Block 3
Data Augmentation
'''

### gamma correction
def gamma_correction(image, gamma):
  invGamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")

  new_img = cv2.LUT(image, table)
  return new_img

### augmentation method 1 - horizontal flip
def horizontal_flip(img):
  img = cv2.flip(img, 0)
  return img

### augmentation method 2 - vertical flip
def vertical_flip(img):
  img = cv2.flip(img,1)
  return img

### augmentation method 3 - Histogram Equalization
def hist(img):
  img_to_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
  img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])
  hist_equalization_result = cv2.cvtColor(img_to_yuv, cv2.COLOR_YUV2BGR)
  return hist_equalization_result

### augmentation method 4 - rotation
def rotation(img):
  h,w = img.shape[:2]
  rotation_matrix = cv2.getRotationMatrix2D((w/2,h/2),45,0.5)
  rotated_img = cv2.warpAffine(img,rotation_matrix,(w,h))
  return rotated_img

def rotation2(img):
  h,w = img.shape[:2]
  rotation_matrix = cv2.getRotationMatrix2D((w/2,h/2),135,0.5)
  rotated_img = cv2.warpAffine(img,rotation_matrix,(w,h))
  return rotated_img

def rotation3(img):
  h,w = img.shape[:2]
  rotation_matrix = cv2.getRotationMatrix2D((w/2,h/2),235,0.5)
  rotated_img = cv2.warpAffine(img,rotation_matrix,(w,h))
  return rotated_img

### augmentation method 5 - increase brightness
def increase_brightness(img):
  bright = np.ones(img.shape, dtype='uint8')*70
  increase_brightness = cv2.add(img,bright)
  return increase_brightness
def increase_brightness2(img):
  bright = np.ones(img.shape, dtype='uint8')*100
  increase_brightness = cv2.add(img,bright)
  return increase_brightness
def increase_brightness3(img):
  bright = np.ones(img.shape, dtype='uint8')*140
  increase_brightness = cv2.add(img,bright)
  return increase_brightness

### augmentation method 6 - decrease brightness
def decrease_brightness(img):
  bright = np.ones(img.shape, dtype='uint8')*70
  decrease_brightness = cv2.subtract(img,bright)
  return decrease_brightness
def decrease_brightness2(img):
  bright = np.ones(img.shape, dtype='uint8')*140
  decrease_brightness = cv2.subtract(img,bright)
  return decrease_brightness
def decrease_brightness3(img):
  bright = np.ones(img.shape, dtype='uint8')*140
  decrease_brightness = cv2.subtract(img,bright)
  return decrease_brightness

In [4]:
'''
Code Block 4
Detecting the faces from an image.
Return a list containing only faces, each face image size is 50x50.
'''

def face_extract_function(image):
  res_face_list = []
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
  faces = face_cascade.detectMultiScale(gray, 1.1, 7)
  for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h),
                  (0, 0, 255), 2)
    face = image[y:y + h, x:x + w]
    resized_face = cv2.resize(face, (50, 50))
    res_face_list.append(resized_face)
  return res_face_list

In [5]:
'''
Code Block 5
'''
categories = ['Abdullah Khan', 'Akshat Kalra', 'Bowen Cheng', 'Chakrapani Chitnis', 'Chin Fang Hsu',
              'Edison Nalluri', 'Kai Cong', 'Kyle Fenole', 'Landis Fusato', 'Minghao Zhang', 'Mohit Jawale',
              'Patrick Lee', 'Peiyuan Li', 'Rahim Firoz Chunara', 'Rohan Vikas Lagare', 'Sadwi Kandula',
              'Samuel Anderson', 'Shaunak Chaudhary', 'Tampara Venkara Santosh Anish Dora', 'Weixuan Lin',
              'Xinyu Dong', 'Yaoyao Peng', 'Yufan Lin']

In [ ]:
'''
Code Block 6
1. Processing the image, read the image from the drive
2. Crop the image, leave only student faces.
3. Convert the image to a numpy array.
4. Dump the image data to a file as dataset.
'''

data = []
dir = '/content/drive/MyDrive/COEN 240/Project/Photos/' # ----- Need to switch this directory path to where your Training photo saved. -----
counter = 0
# constructing the data structure for all student pictures.
for category in categories: #category is student name
  path = dir + category
  image_dir = Path(path)
  label = categories.index(category)
  image_dir_list = image_dir.glob("*.*") # read the image in all type, *.png and 8.jpg
  for image in image_dir_list:
    counter += 1
    student_image = cv2.imread(str(image))
    student_face_extract = face_extract_function(student_image)
    for img in student_face_extract:
      #normalization
      img = hist(img)

      original_img = np.array(img).flatten()
      data.append([original_img, label])

      #horizontal_flip_img = np.array(horizontal_flip(img)).flatten()
      #data.append([horizontal_flip_img, label])

      #vertical_flip_img = np.array(vertical_flip(img)).flatten()
      #data.append([vertical_flip_img, label])

      #Decrease accuracy
      rotated_img = np.array(rotation(img)).flatten()
      data.append([rotated_img, label])
      rotated_img2 = np.array(rotation2(img)).flatten()
      data.append([rotated_img2, label])
      rotated_img3 = np.array(rotation3(img)).flatten()
      data.append([rotated_img3, label])

      increase_brightness_img = np.array(increase_brightness(img)).flatten()
      data.append([increase_brightness_img, label])
      increase_brightness_img2 = np.array(increase_brightness2(img)).flatten()
      data.append([increase_brightness_img2, label])
      increase_brightness_img3 = np.array(increase_brightness3(img)).flatten()
      data.append([increase_brightness_img3, label])

      decrease_brightness_img = np.array(decrease_brightness(img)).flatten()
      data.append([decrease_brightness_img, label])
      decrease_brightness_img2 = np.array(decrease_brightness2(img)).flatten()
      data.append([decrease_brightness_img2, label])
      decrease_brightness_img3 = np.array(decrease_brightness3(img)).flatten()
      data.append([decrease_brightness_img3, label])

      #gamma correction - gamma value = 0.5
      np.array(decrease_brightness2(img)).flatten()
      gamma_correction_img = np.array(gamma_correction(img, 0.5)).flatten()
      data.append([gamma_correction_img, label])

      #gamma correction - gamma value = 0.25
      np.array(decrease_brightness2(img)).flatten()
      gamma_correction_img = np.array(gamma_correction(img, 0.25)).flatten()
      data.append([gamma_correction_img, label])

      #gamma correction - gamma value = 0.75
      np.array(decrease_brightness2(img)).flatten()
      gamma_correction_img = np.array(gamma_correction(img, 0.75)).flatten()
      data.append([gamma_correction_img, label])

pick_in = open('/content/drive/MyDrive/COEN 240/Project/dataset.pickle', 'wb')
pickle.dump(data, pick_in)
pick_in.close()


In [6]:
'''
Code Block 7
'''
### Load the testing data

dir = '/content/drive/MyDrive/COEN 240/Project/Testing_Photos/' # ----- Need to switch this directory path to where your Testing photo saved. -----
# constructing the data structure for all student pictures.
data = []
for category in categories: #category is student name
  path = dir + category
  image_dir = Path(path)
  label = categories.index(category)
  image_dir_list = image_dir.glob("*.*") # read the image in all type, *.png and 8.jpg
  for image in image_dir_list:
    student_image = cv2.imread(str(image))
    student_face_extract = face_extract_function(student_image)
    for img in student_face_extract:
      original_img = np.array(img).flatten()
      data.append([original_img, label])

pick_in = open('/content/drive/MyDrive/COEN 240/Project/test_dataset.pickle', 'wb')
pickle.dump(data, pick_in)
pick_in.close()


In [7]:
'''
Code Block 8
'''
### Load the Training dataset
pick_in = open('/content/drive/MyDrive/COEN 240/Project/dataset.pickle', 'rb')
data = pickle.load(pick_in)
pick_in.close()

random.shuffle(data)
X_train = []
y_train = []
for feature, label in data:
  feature = feature.flatten()
  X_train.append(feature)
  y_train.append(label)

In [8]:
'''
Code Block 9
'''
### Load testing dataset
pick_in = open('/content/drive/MyDrive/COEN 240/Project/test_dataset.pickle', 'rb')
data = pickle.load(pick_in)
pick_in.close()

X_test = []
y_test = []
for feature, label in data:
  feature = feature.flatten()
  X_test.append(feature)
  y_test.append(label)


In [9]:
'''
Code Block 10
'''
### Feature extraction for training the model
pca = PCA(n_components=100)
pca.fit(X_train)

PCA(n_components=100)

In [10]:
'''
Code Block 11
'''
### Transform X_train and X_test to the designate shape
X_train = pca.transform(X_train)
X_test_data = pca.transform(X_test)


In [11]:
'''
Code Block 12
'''
# SVM
svm_model = SVC(C = 1000, gamma = 'auto', kernel = 'rbf')
svm_model.fit(X_train, y_train)

SVC(C=1000, gamma='auto')

In [12]:
'''
Code Block 13
'''
predictions = svm_model.predict(X_test_data)
print(accuracy_score(y_test, predictions))

0.09722222222222222


In [ ]:
'''
Code Block 14
'''
# K-Fold Cross-Validation
from sklearn.model_selection import cross_validate
def cross_validation(model, _X, _y, _cv = 10):
  results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring='accuracy',
                               return_train_score=True)
  print("Mean Training Accuracy:", results['train_score'].mean()*100)
  print("Mean Testing Accuracy:", results['test_score'].mean()*100)

In [ ]:
'''
Code Block 15
'''
# KNN (n=1 has better average accuracy than 3 and 5)
knn_model = KNeighborsClassifier(n_neighbors=1)
cross_validation(knn_model,X_train,y_train)
knn_model2 = KNeighborsClassifier(n_neighbors=1)
cross_validation(knn_model2,X_train,y_train)
knn_model3 = KNeighborsClassifier(n_neighbors=3)
cross_validation(knn_model3,X_train,y_train)
knn_model4 = KNeighborsClassifier(n_neighbors=5)
cross_validation(knn_model4,X_train,y_train)


Mean Training Accuracy: 100.0
Mean Testing Accuracy: 87.93821343752396
Mean Training Accuracy: 100.0
Mean Testing Accuracy: 87.93821343752396
Mean Training Accuracy: 91.75892718210838
Mean Testing Accuracy: 77.58101585842336
Mean Training Accuracy: 85.05243378584193
Mean Testing Accuracy: 74.040258944304


In [13]:
'''
Code Block 16
'''
# KNN (n=1 has better average accuracy than 3 and 5)
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [14]:
'''
Code Block 17
'''
predictions = knn_model.predict(X_test_data)
print(accuracy_score(y_test, predictions))

0.6527777777777778


In [15]:
'''
Code Block 18
'''
### Decision tree
decision_tree_model = DecisionTreeClassifier(random_state=0)
decision_tree_model.fit(X_train, y_train)


DecisionTreeClassifier(random_state=0)

In [16]:
'''
Code Block 19
'''
predictions = decision_tree_model.predict(X_test_data)
print(accuracy_score(y_test,predictions))

0.3333333333333333


In [ ]:
'''
Code Block 20
'''
# Random forest
# limit max depth to prevent overfitting
random_forest_model = RandomForestClassifier(class_weight = 'balanced', random_state=0, max_depth = 15)

cross_validation(random_forest_model,X_train,y_train)

random_forest_model2 = RandomForestClassifier( class_weight = 'balanced', random_state=0, max_depth = 8)
cross_validation(random_forest_model2,X_train,y_train)

random_forest_model3 = RandomForestClassifier( class_weight = 'balanced', random_state=0, max_depth = 10)
cross_validation(random_forest_model3,X_train,y_train)

random_forest_model4 = RandomForestClassifier( class_weight = 'balanced', random_state=0, max_depth = 12)
cross_validation(random_forest_model4,X_train,y_train)

random_forest_model5 = RandomForestClassifier( class_weight = 'balanced', random_state=0, max_depth = 15)
cross_validation(random_forest_model5,X_train,y_train)


Mean Training Accuracy: 100.0
Mean Testing Accuracy: 86.66992262315176
Mean Training Accuracy: 96.68808970326815
Mean Testing Accuracy: 82.2561863173217
Mean Training Accuracy: 99.67948433823565
Mean Testing Accuracy: 84.31012027886308
Mean Training Accuracy: 99.97571872480279
Mean Testing Accuracy: 85.7088408794913
Mean Training Accuracy: 100.0
Mean Testing Accuracy: 86.66992262315176


In [44]:
'''
Code Block 21
'''
### Random forest
random_forest_model = RandomForestClassifier( class_weight = 'balanced', random_state=0,max_depth = 15, n_estimators=400)
random_forest_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=15, n_estimators=400,
                       random_state=0)

In [46]:
'''
Code Block 22
'''
predictions = random_forest_model.predict(X_test_data)
print(accuracy_score(y_test, predictions))

0.5416666666666666
